In [82]:
import time
query =  "большой театр"

In [83]:
from elasticsearch import Elasticsearch

es = Elasticsearch()
t = time.time()
res = es.search(index="items-index3",
                body={"query": {"simple_query_string": {
                    'query': query,
                    'fields': [ 'name^2', 'annotation' ]
          }}})
print("Найдено %d записей за %0.3fс:" % (res['hits']['total'], time.time()-t))

for hit in res['hits']['hits'][:10]:
    print(hit["_id"],'\t',hit["_score"],'\t',hit["_source"]["name"])

Найдено 245381 записей за 0.428с:
25187953 	 55.79111 	 Большой театр  
20467618 	 54.35475 	 Большой театр  
17922732 	 53.739975 	 Наш Большой театр  
2659641 	 52.498142 	 Трупы Большого театра  
32876879 	 52.41837 	 Наш Большой театр  
7113590 	 52.401215 	 Трупы Большого театра  
3752910 	 52.294712 	 Художники Большого театра  
2890717 	 52.153015 	 Большой Драматический Театр  
19119731 	 51.889175 	 Большой театр "Малышка"  
6132303 	 51.21914 	 Картина-гравюра большая "Большой театр"  


In [84]:
import psycopg2
conn = psycopg2.connect(dbname="postgres", host="localhost", user="postgres", password="111")
cur = conn.cursor()
t = time.time()
cur.execute(
    "SELECT id, name, ts_rank(make_tsvector(name,annotation),q) as rank"+
    " FROM items, plainto_tsquery('russian','"+query+"') AS q"+
    " WHERE make_tsvector(name,annotation) @@ q"+
    " ORDER BY ts_rank(make_tsvector(name,annotation),q) DESC"+
    " limit 10;")
print("Найдено за %0.3fс:" % (time.time()-t))
for id_,name, rank in cur.fetchall():
    print(id_,'\t',rank,'\t',name)
cur.close()
conn.close()

Найдено за 43.800с:
6132303 	 0.999995 	 Картина-гравюра большая "Большой театр"  
27030530 	 0.999992 	 Оперная труппа Большого театра  
3146998 	 0.999986 	 История русского балета. Мариинский театр. Большой театр  
3237308 	 0.999982 	 Большой театр (Москва). Пазл, 1000 элементов  
24929395 	 0.999953 	 Большой Театр СССР. Опера. Балет  
6077685 	 0.999933 	 Большой Театр СССР (комплект из 4 книг)  
2488560 	 0.999929 	 Календарь настенный перекидной на 2006. Большой театр  
3562005 	 0.9999 	 Большой театр СССР. История сооружения и реконструкции здания  
5629128 	 0.999894 	 Большой театр СССР: опера, балет  
19064032 	 0.999819 	 CubicFun "Большой театр", 29 элементов  


In [85]:
from sphinxapi import *
cl = SphinxClient()
cl.SetServer ( '35.234.137.224', 9312 )
cl.SetMatchMode ( SPH_MATCH_ALL )
cl.SetLimits ( 0, 10 )
res = cl.Query ( query, 'test1' )
cl.SetSortMode(SPH_SORT_RELEVANCE)
print('Найдено {} записей за {}с:'.format(res['total_found'],res['time']))
for rec in res['matches']:
    print(rec['id'],'\t',rec['weight'])
print(res['words'])

Найдено 3057 записей за 0.128с:
68332 	 4
73615 	 4
154978 	 4
1214580 	 4
1270954 	 4
1387794 	 4
2076490 	 4
2170711 	 4
2183610 	 4
2364686 	 4
[{'word': 'больш', 'docs': 283289, 'hits': 345960}, {'word': 'театр', 'docs': 19980, 'hits': 30828}]
